## Selenium 활용 데이터 수집 - 네이버 웹툰 목록 가져오기

body 안에 아무것도 없는 경우 , 인스타그램 등 class이름이 접속할때마다 바뀌는 경우 : 
-> 자바스크립트로 html에 들어갈 내용을 생성함. 리액트를 사용하면 저렇다. 이러면 request로 가져오지 못하고 개발자 도구에서 가져와야 하는데 이는 실제로 웹에서 동작을 해야 실행되기 때문이다. 

<셀리니움 파이썬 라이브러리>
셀리니움은 chrome-driver.exe 를 제어하는데 이는 크롬 브라우저를 제어한다. 
1) 특정 페이지 접속
2) Element code 가져오기
3) 특정 요소 input에 키보드 입력 보내기
4) 특정 요소에 마우스 클릭하기 
5) 자바스크립트 코드 강제 실행 -> 스크롤 발생

In [22]:
import chromedriver_autoinstaller #ChromeDriver 자동 설치 모듈
from selenium import webdriver # Chrome을 제어하기 위한 객체
from selenium.webdriver.support.ui import WebDriverWait # Chrome이 웹페이지 로딩을 완료할 때까지 최대 n초간 대기하는 기능
from bs4 import BeautifulSoup
from pandas import DataFrame 
import time # 지정된 시간동안 파이썬에 렉을 거는 기능 

### 크롬 브라우저 가동하기

In [23]:
# 크롬 드라이버 자동 설치
chromedriver_autoinstaller.install()

# 크롬드라이버를 통해 크롬을 실행시킴
driver = webdriver.Chrome() # driverㄴ 객체는 Chrome 자체 

# 크롬 브라우저가 준비될 때까지 최대 5초씩 대기해라 
driver.implicitly_wait(5)


# 새 창이 뜨더라도 사용하면 안된다. 파이썬 코드 엉킴

In [27]:

# 요일별 네이버 웹툰에 접근하기 위한 변수값 ULR을 분석하여 얻어낸 값
params = ['mon','tue','wed','thu','fri','sat','sun','dailyPlus']

# 네이버 웹툰의 주소 형식
naverWebtoonUrl = 'https://comic.naver.com/webtoon?tab={0}'

# 수집된 결과가 누적될 빈 리스트
naverWebtoonData = []


# params 를 가지고 반복을 돌려서 mon,tue 등이 들어가게 된다. ?tab={p} 에 들어감 
# 이 url을 뒤에 떠있는 크롬으로 보낸다.
# 드라이버는 크롬 그 자체라서 리스트에 잇는 숫자만큼 8번 바뀐다. 

# 요일별 반복 
for p in params :
    url = naverWebtoonUrl.format(p)  # 특정 요일의 네이버 웹툰 페이지
    # print(url) # https://comic.naver.com/webtoon?tab=mon
    driver.get(url) # 크롬브라우저로 방문시킨다.
    time.sleep(1)

    # 브라우저에 표시되는 전체 코드를 추출한다. 
    # driver.page_source # 이게 r.text와 같다. 화면에 보이는 html코드를 쭉 가져온다. 이 코드를 뷰티풀솦에게 보낸다. 우리가 알 수 있게
    soup = BeautifulSoup(driver.page_source)
    # print(soup)

    # 크롬브라우저로 띄운건 건드리면 안되니까 새로 띄워서 html 확인해라 
    
    # 웹툰이 표시되는 부분만 추출하기
    webtoonList = soup.select('.ContentList__content_list--q5KXY > .item')
    # print(webtoonList) # select는 항상 list로 반환



    # 추출된 웹툰 목록 수 만큼 반복한다. 
    # 즉 요일별로 반복을 하면서, 내부에서는 웹툰 목록 수 만큼 반복한다.
    for w in webtoonList :
        poster = w.select('.Poster__image--d9XTI') # 포스터 url 가져오기
        if poster and "src" in poster[0].attrs :
            posterValue = poster[0].attrs['src']
        else :
            posterValue = None 
        #print(posterValue)

        # 웹툰의 url 가져오기
        url = w.select("Poster__link--sopnC")

# 검색결과에는 나오는데 포스터가 없는 애들이 있어서 if문이 필요하다. (아직 런칭 전인 웹툰들임)
# 웹툰 주소를 가져오기 위해서 "Poster_link--sopnC" class를 긁어온다. 그리고 a href=/..'주소를 가져온다.
# if문을 꼼꼼하게 주는 이유는 예측하지 못하는 일들이 발생할 수 있기 때문이다. 

        if url and 'href' in url[0].attrs :
            urlValue = url[0].attrs['href'] 

            if urlValue.find('https://comic.naver.com') == -1 : # 앞에 사이트 주소가 없으면 붙여줘
                urlValue = 'https://comic.naver.com' + urlValue
        else :
            urlValue = None 

        # 웹툰 제목 가져오기
        title = w.select('.ContentTitle__title--e3qXt > .text')
        if title :
            titleValue = title[0].text.strip()
        else :
            titleValue : None 

        # 작가 이름 가져오기
        author = w.select('.ContentAuthor__author--CTAAP')
        if author :
            authorValue = author[0].text.strip()
        else :
            authorValue = None
        print(authorValue)

        # 별점 가져오기 
        rating = w.select('.Rating__star_area--dFzsb > .text')
        if rating :
            ratingValue = rating[0].text.strip()
        else :
            ratingValue = None 
        
        # 결과 병합

        resultDic = {'요일':p,"제목":titleValue,'작가':authorValue,'별점':ratingValue}
        print(resultDic)
        naverWebtoonData.append(resultDic)

df = DataFrame(naverWebtoonData)
df.to_excel('요일별_네이버_웹툰.xlsx')
df





[<li class="item"><a class="Poster__link--sopnC" href="/webtoon/list?titleId=648419"><div class="Poster__thumbnail_area--gviWY Poster__type160x207--EQAM8"><div class="Poster__bullet_wrap--VcWFJ"></div><span class="Poster__image--d9XTI" style="display: inline-block;"></span></div></a><div class="ContentList__info_area--bXx7h"><a class="ContentTitle__title_area--x24vt" href="/webtoon/list?titleId=648419"><i class="ContentTitle__icon_bullet--H1qaP"><svg aria-hidden="true" fill="none" height="13" width="17" xmlns="http://www.w3.org/2000/svg"><rect height="12" rx="1.5" stroke="#FF4747" width="16" x="0.5" y="0.5"></rect><path d="M3.449 6.899c0 1.518.957 2.53 2.401 2.53 1.454 0 2.42-1.012 2.42-2.53V3.403H7.12V6.89c0 .837-.497 1.389-1.251 1.389-.764 0-1.27-.561-1.27-1.399V3.403h-1.15v3.496Zm6.174 2.4h1.15V7.314h.929c1.435 0 2.383-.773 2.383-1.96 0-1.177-.939-1.95-2.365-1.95H9.623V9.3Zm2.19-4.884c.634 0 1.057.377 1.057.938 0 .57-.423.948-1.058.948h-1.04V4.415h1.04Z" fill="#FF4747"></path></svg>

,요일,제목,작가,별점
0,mon,뷰티풀 군바리,설이 / 윤성원,9.81
1,mon,신의 탑,SIU,9.88
2,mon,퀘스트지상주의,박태준 만화회사,9.76
3,mon,똑 닮은 딸,이담,9.98
4,mon,백수세끼,치즈,9.84
...,...,...,...,...
721,dailyPlus,후드,후렛샤 / 심규호,8.27
722,dailyPlus,실버벨,노떼,9.53
723,dailyPlus,씨앗의 정원,백희,9.69
724,dailyPlus,도와줘우주,설레임 / 한겨울,9.33
